In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import uproot

In [2]:
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

In [3]:
path_to_data = '../data/'

folder15 = 'breynold/user.breynold.data15_13TeV.00284484.physics_Main.DAOD_NTUP_JTRIG_JETM1.r9264_p3083_p3601_j042_tree.root/'
file15 = 'user.breynold.18753218._000001.tree.root'
folder16 = 'breynold/user.breynold.data16_13TeV.00307656.physics_Main.DAOD_NTUP_JTRIG_JETM1.r9264_p3083_p3601_j042_tree.root/'
file16 = 'user.breynold.18797259._000001.tree.root'

folder = 'doglioni/'
file = 'data18_13TeV.00355331.calibration_DataScouting_05_Jets.merge.AOD.r10657_p3592.root'
file = 'dijetISRphoton.root'
file = 'dijetISRphoton36k.root'

# Load a ROOT file
filePath = path_to_data + folder16 + file16
#filePath = path_to_data + folder + file
ttree = uproot.open(filePath)['outTree']['nominal']
#ttree = uproot.open(filePath)['CollectionTree']

In [4]:
ttree.keys()

[b'runNumber',
 b'eventNumber',
 b'lumiBlock',
 b'coreFlags',
 b'bcid',
 b'NPV',
 b'actualInteractionsPerCrossing',
 b'averageInteractionsPerCrossing',
 b'weight_pileup',
 b'correctedAverageMu',
 b'correctedAndScaledAverageMu',
 b'correctedActualMu',
 b'correctedAndScaledActualMu',
 b'passL1',
 b'passHLT',
 b'passedTriggers',
 b'disabledTriggers',
 b'triggerPrescales',
 b'isPassBits',
 b'isPassBitsNames',
 b'nAntiKt4EMTopoJets_Calib2016',
 b'AntiKt4EMTopoJets_Calib2016_E',
 b'AntiKt4EMTopoJets_Calib2016_pt',
 b'AntiKt4EMTopoJets_Calib2016_phi',
 b'AntiKt4EMTopoJets_Calib2016_eta',
 b'AntiKt4EMTopoJets_Calib2016_clean_passLooseBad',
 b'AntiKt4EMTopoJets_Calib2016_clean_passTightBad',
 b'AntiKt4EMTopoJets_Calib2016_JvtPass_Medium',
 b'nAntiKt4EMTopoJets_Calib2018',
 b'AntiKt4EMTopoJets_Calib2018_E',
 b'AntiKt4EMTopoJets_Calib2018_pt',
 b'AntiKt4EMTopoJets_Calib2018_phi',
 b'AntiKt4EMTopoJets_Calib2018_eta',
 b'AntiKt4EMTopoJets_Calib2018_clean_passLooseBad',
 b'AntiKt4EMTopoJets_Calib201

In [5]:
branchnames = ['nAntiKt4EMTopoJets_Calib2018',
               'AntiKt4EMTopoJets_Calib2018_E',
               'AntiKt4EMTopoJets_Calib2018_pt',
               'AntiKt4EMTopoJets_Calib2018_phi',
               'AntiKt4EMTopoJets_Calib2018_eta']


#branchnames = ['nDSTrigJet',
#               'DSTrigJet_E',
#              'DSTrigJet_pt',
#              'DSTrigJet_phi',
#              'DSTrigJet_eta']

#branchnames = ['n_jet',
#               'jet_e',
#               'jet_pt',
#               'jet_phi',
#               'jet_eta',
#               'n_photons',
#               'photons_e',
#               'photons_pt',
#               'photons_phi',
#               'photons_eta']

jaggedE = ttree.array(branchnames[1])
jaggedpT = ttree.array(branchnames[2])
jaggedphi = ttree.array(branchnames[3])
jaggedeta = ttree.array(branchnames[4])

In [6]:
jaggedE_ph = ttree.array(branchnames[6])
jaggedpT_ph = ttree.array(branchnames[7])
jaggedphi_ph = ttree.array(branchnames[8])
jaggedeta_ph = ttree.array(branchnames[9])

IndexError: list index out of range

In [7]:
e_list = []
pt_list = []
eta_list = []
phi_list = []
for ii, arr in enumerate(jaggedE):
    for kk, val in enumerate(arr):
        e_list.append(val)
        pt_list.append(jaggedpT[ii][kk])
        eta_list.append(jaggedeta[ii][kk])
        phi_list.append(jaggedphi[ii][kk])


all_e = np.array(e_list)
all_pt = np.array(pt_list)
all_eta = np.array(eta_list)
all_phi = np.array(phi_list)

In [8]:
print(all_e.shape, all_eta.shape, all_phi.shape, all_pt.shape)

(22621866,) (22621866,) (22621866,) (22621866,)


In [ ]:
def get_leading(jaggedX):
    return jaggedX[jaggedX.counts > 0, 0]

In [ ]:
leading_E = get_leading(jaggedE)
leading_pT = get_leading(jaggedpT)
leading_phi = get_leading(jaggedphi)
leading_eta = get_leading(jaggedeta)

In [ ]:
leading_ph_E = get_leading(jaggedE_ph)
leading_ph_pT = get_leading(jaggedpT_ph)
leading_ph_phi = get_leading(jaggedphi_ph)
leading_ph_eta = get_leading(jaggedeta_ph)

In [ ]:
print(leading_E.shape, leading_eta.shape, leading_phi.shape, leading_pT.shape)

In [9]:
df = pd.DataFrame(data = {'jet pT': all_pt, 'jet eta': all_eta, 'jet phi': all_phi, 'jet E': all_e})

In [10]:
df.head()

,jet pT,jet eta,jet phi,jet E
0,161.850494,-0.764774,2.287350,212.217300
1,155.803253,-0.482176,-0.687477,175.156143
2,29.825865,-1.400844,-0.953364,64.451935
3,22.049797,1.565698,-2.697407,55.199814
4,21.709501,-0.085313,0.009056,22.383587


In [ ]:
df1.head()

In [ ]:
df2.head()

In [11]:
train, test = train_test_split(df, test_size=0.2, random_state=41)

In [ ]:
print(train.shape, test.shape)

In [17]:
# Save train and test sets
#train.to_pickle('processed_data/all_jets_train.pkl')
#test.to_pickle('processed_data/all_jets_test.pkl')
#train = pd.read_pickle('processed_data/all_jets_train.pkl')
#test = pd.read_pickle('processed_data/all_jets_test.pkl')